# Accuracy across 20 rows at 10M and 100M paths

In [ ]:

"""

Checks:
1. Statistical significance between 10M and 100M runs using MC-SE of the *mean*.
2. Whether the two runs match to 3 dp (|Δmean| < tol).
3. Whether each run’s own SE is small enough (SE_of_mean < tol).
"""

import re
import pandas as pd
import numpy as np
from math import sqrt

# ── edit these paths if needed ──────────────────────────────────
FILE_10M  = "Data/Test10.parquet"
FILE_100M = "Data/Test100.parquet"


# ── knobs ───────────────────────────────────────────────────────
ABS_TOL = 5e-4       # tolerance on mean difference for 3dp
Z_CRIT  = 1.96       # two-tailed z-critical for α = 0.05


def load_dfs():
    df10 = pd.read_parquet(FILE_10M)
    df100 = pd.read_parquet(FILE_100M)
    return df10, df100


def metrics_from_df(df):
    """
    Identify all metrics that have a matching SE column in the dataframe.
    """
    metrics = set()
    for col in df.columns:
        m = re.match(r"(.+)_se(?:_(\d+))?", col)
        if m:
            prefix, idx = m.group(1), m.group(2)
            metric = f"{prefix}_{idx}" if idx else prefix
            metrics.add(metric)
    return sorted(metrics)


def compute_run_se_of_mean(se_series):
    """
    Compute SE of the mean across rows:
      SE_mean = sqrt(sum(se_i**2)) / N_rows
    """
    return np.sqrt((se_series.values**2).sum()) / len(se_series)


def compare(df_a, df_b, tol=ABS_TOL, zcrit=Z_CRIT):
    mets = metrics_from_df(df_a)
    rows = []
    for m in mets:
        # determine se column name
        if '_' in m:
            base, idx = m.rsplit('_',1)
            se_col = f"{base}_se_{idx}"
        else:
            se_col = f"{m}_se"

        try:
            # compute means
            ma = df_a[m].mean()
            mb = df_b[m].mean()
            # compute SE of mean
            sea = compute_run_se_of_mean(df_a[se_col])
            seb = compute_run_se_of_mean(df_b[se_col])
        except Exception:
            # missing columns or other error: skip
            continue

        # skip if any is not finite
        if not all(np.isfinite([ma, mb, sea, seb])):
            continue

        # z-test for mean difference
        diff       = ma - mb
        pooled_se  = sqrt(sea**2 + seb**2)
        zscore     = diff / pooled_se
        significant= abs(zscore) > zcrit

        # 3dp agreement
        same_3dp   = abs(diff) < tol
        # intrinsic accuracy
        acc_a      = sea < tol
        acc_b      = seb < tol

        rows.append({
            "metric":         m,
            "mean_10M":       ma,
            "se_mean_10M":    sea,
            "mean_100M":      mb,
            "se_mean_100M":   seb,
            "diff":           diff,
            "pooled_se":      pooled_se,
            "z":              zscore,
            "significant?":   significant,
            "same_3dp?":      same_3dp,
            "accurate_10M?":  acc_a,
            "accurate_100M?": acc_b,
        })
    return pd.DataFrame(rows).set_index("metric")


def main():
    df10, df100 = load_dfs()
    cmp = compare(df10, df100)

    # PRICE diagnostics
    print("\nPRICE diagnostics:\n")
    print(cmp.loc["price", [
        "mean_10M","se_mean_10M",
        "mean_100M","se_mean_100M",
        "diff","pooled_se","z",
        "significant?","same_3dp?",
        "accurate_10M?","accurate_100M?"
    ]])

    # Full comparison table
    pd.set_option("display.float_format", "{:0.6f}".format)
    print("\nFull comparison table:\n")
    print(cmp)

    # Accuracy summary
    print("\nACCURACY SUMMARY:\n")
    match3 = cmp.index[cmp["same_3dp?"]].tolist()
    fail3  = cmp.index[~cmp["same_3dp?"]].tolist()
    sig    = cmp.index[cmp["significant?"]].tolist()
    acc10  = cmp.index[cmp["accurate_10M?"]].tolist()
    acc100 = cmp.index[cmp["accurate_100M?"]].tolist()

    print(f"Metrics matching 3dp       : {', '.join(match3) or 'None'}")
    print(f"Metrics failing  3dp       : {', '.join(fail3) or 'None'}")
    print(f"Significantly different    : {', '.join(sig) or 'None'}")
    print(f"Accurate at 3dp (10M run)  : {', '.join(acc10) or 'None'}")
    print(f"Accurate at 3dp (100M run) : {', '.join(acc100) or 'None'}")

if __name__ == "__main__":
    main()



PRICE diagnostics:

mean_10M           29.98516
se_mean_10M        0.004453
mean_100M         29.987016
se_mean_100M       0.001404
diff              -0.001856
pooled_se          0.004669
z                 -0.397579
significant?          False
same_3dp?             False
accurate_10M?         False
accurate_100M?        False
Name: price, dtype: object

Full comparison table:

           mean_10M  se_mean_10M   mean_100M  se_mean_100M        diff  \
metric                                                                   
delta_0    0.083708     0.141697   -0.013803      0.045151    0.097511   
delta_1    0.324215     0.191464    0.195029      0.060553    0.129186   
delta_2   -0.107936     0.198003    0.079814      0.062779   -0.187751   
gamma_0  -49.100968    34.012989  -15.084881     10.731792  -34.016087   
gamma_1  -55.424309    60.239530    3.618279     19.039738  -59.042588   
gamma_2  -52.284884    51.743458  -13.938086     16.265926  -38.346799   
price     29.985160     0.0

: 

# Ferguson and Green Check

## Original Code

In [ ]:
import numpy as np, torch
from WOF import fg_sample, price_mc, SEED_BASE

# 1) Fix your seeds for full reproducibility
np.random.seed(SEED_BASE)
torch.manual_seed(SEED_BASE)

# 2) Draw one scenario and compute price + SE
params = fg_sample()
price, se = price_mc(
    params,
    n_paths= 100_000_000,
    n_steps=64,
    return_se=True
)

# 3) Define your accuracy thresholds (in absolute price‐error units)
thresholds = {
    "1 cent (0.01)":    0.01,
    "0.1 cent (0.001)": 0.001,
    "0.01 cent (0.0001)": 0.0001,
}

# 4) Print results
print(f"price = {price:.6f},  SE = {se:.6f}\n")
for label, tol in thresholds.items():
    status = "PASS" if se < tol else "FAIL"
    print(f"{label:15}: {status}  (SE = {se:.6f} {'<' if status=='PASS' else '>'} {tol:.6f})")


Able to replicate the Ferguson and Green of 1 cent accuracy with both 10 Mil and 100 Mil paths

## Variance Reduction Code

## Goal

Find an effective method to drastically reduce the Monte Carlo standard error (variance) for a robust test dataset of worst‑of option payoffs.

---

## Methods Attempted

1. **Sobol/QMC**
   Owen‑scrambled Sobol quasi‑Monte Carlo for low‑discrepancy sampling.

2. **Antithetic Variates**
   Pairing each Sobol point with its antithetic counterpart.

3. **Brownian Bridge**
   Reordering the time increments to concentrate variance in early steps.

4. **Exponential Tilting (Importance Sampling)**
   Tilting the asset Brownian drifts to overweight scenarios where the payoff is nonzero, with likelihood‑ratio correction.

5. **Control Variate: Sum of Vanilla Calls**
   Adding the sum of individual call payoffs as a crude variate (β=1).

6. **Regression‑based Control Variate**
   Estimating the optimal β via sample covariance/variance between the target payoff and the call‑sum variate.

7. **Geometric‑Basket Control Variate**
   Using the analytic geometric‑basket call payoff as a highly correlated variate.

8. **Multi‑Level Monte Carlo (MLMC)**
   Telescoping coarse and fine time‑step estimates to reduce both bias and variance.

---

## Summary of Results

* **Base Monte Carlo** with 100 million Sobol+antithetic+bridge paths: SE ≈ 0.00299.
* **Regression CV** (sum‑of‑calls): SE ≈ 0.00299 (minimal gain over β=1).
* **Geometric‑Basket CV**: moderate improvement but still SE ≳ 0.0025.
* **Exponential Tilting + Regression CV**: introduced NaNs at high path counts; once stabilized, SE ≳ 0.0029.

*No combination so far has achieved SE ≤ 0.001 on 100 M paths.*

---

## Next Steps

* **Fine‑tune Importance Sampling**: search for optimal tilt vector θ to maximize variance reduction.
* **Implement MLMC**: start with a two‑level scheme (e.g. 16 vs 64 timesteps) to cut cost and variance.
* **Explore Stratification**: stratify by asset index or payoff buckets in conjunction with Sobol.
* **Hybrid Methods**: combine tailored tilting, MLMC, and geometric CV for multiplicative gains.

*By iterating on these advanced techniques, we aim for another 5×–10× reduction in SE.*


Have Failed to Replicate this best to probably also switch test set to AAD for Greeks to minimize variance, as well as probably tune down the Validation 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
make_worst_of_dataset_fast_streaming.py
  • multi-GPU, single-process Monte-Carlo in FP32
  • Owen-scrambled Sobol, antithetic, Brownian bridge
  • Streaming Welford algorithm for regression control variate
  • Geometric-basket control variate with analytic price
  • Chunk-safe up to 100 M paths on 4×12 GiB GPUs
  • O(1) RAM / no large tensor concatenations
"""

import os
import math
import sys
import numpy as np
import torch
from torch.distributions import Beta, Normal
from torch.quasirandom import SobolEngine

torch.set_default_dtype(torch.float32)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark       = True

# Constants
N_ASSETS   = 3
R_RATE     = 0.03
SEED_BASE  = 42
CHUNK_PATH = 1_000_000    # paths per chunk per GPU

# Devices
NGPU    = torch.cuda.device_count()
DEVICES = [torch.device(f"cuda:{i}") for i in range(NGPU)]
if NGPU == 0:
    sys.exit("No CUDA GPU visible – aborting.")

# Correlation sampler

def cvine_corr(d, a=5.0, b=2.0):
    beta = Beta(torch.tensor([a], device="cuda"), torch.tensor([b], device="cuda"))
    P    = torch.eye(d, device="cuda")
    for k in range(d-1):
        for i in range(k+1, d):
            rho = 2*beta.sample().item() - 1.0
            for m in range(k-1, -1, -1):
                rho = rho*math.sqrt((1-P[m,i]**2)*(1-P[m,k]**2)) + P[m,i]*P[m,k]
            P[k,i] = P[i,k] = rho
    ev,evec = torch.linalg.eigh(P)
    return evec @ torch.diag(torch.clamp(ev,min=1e-6)) @ evec.T

# Sample generator

def fg_sample():
    z = np.random.normal(0.5, 0.5, N_ASSETS)
    return dict(
        S0    = (100*np.exp(z)).astype(np.float32),
        sigma = np.random.uniform(0.0,1.0,N_ASSETS).astype(np.float32),
        T     = float((np.random.randint(1,44)**2)/252.0),
        rho   = cvine_corr(N_ASSETS).cpu().numpy().astype(np.float32),
        K     = 100.0,
        r     = R_RATE
    )

# Brownian bridge

def brownian_bridge(Z):
    order = [Z.shape[1]-1] + list(range(Z.shape[1]-1))
    return Z[:,order,:]

# QMC+antithetic path generator

def generate_qmc_paths(engine, m, n_steps, d, device):
    u = engine.draw(m//2, dtype=torch.float32)
    u = torch.cat([u, 1.0-u], dim=0).to(device)
    u = u.clamp(min=1e-6, max=1-1e-6)
    normals = Normal(0.,1.).icdf(u).view(m, n_steps, d)
    return brownian_bridge(normals)

# Chunk payoff generator (returns discounted worst-of & geo payoffs)
@torch.no_grad()
def chunk_payoffs(params, m, n_steps, engine, device):
    Z     = generate_qmc_paths(engine, m, n_steps, N_ASSETS, device)
    S0    = torch.tensor(params['S0'],   device=device)
    sigma = torch.tensor(params['sigma'],device=device)
    T     = torch.tensor(params['T'],    device=device)
    rho   = torch.tensor(params['rho'],  device=device)
    K,r   = params['K'], params['r']

    dt    = T / n_steps
    mu    = r - 0.5 * sigma**2
    sig   = sigma
    chol  = torch.linalg.cholesky(rho)

    logS = torch.log(S0).expand(m, N_ASSETS).clone()
    sqrt_dt = math.sqrt(dt.item())
    for k in range(n_steps):
        dW   = Z[:,k,:] @ chol.T
        logS = logS + mu*dt + sig*sqrt_dt*dW
    ST      = torch.exp(logS)
    payoff  = torch.clamp(ST.min(dim=1).values - K, 0.)
    geo_pay = torch.clamp(torch.exp(logS.mean(dim=1)) - K, 0.)

    disc_f = math.exp(-r * T.item())
    return disc_f * payoff.cpu(), disc_f * geo_pay.cpu()

# Analytic geometric basket call price

def geo_call_price(S0, K, r, T, sigma, rho):
    # sigma_G^2 = (1/d^2) * sigma^T rho sigma
    vec = torch.tensor(sigma, dtype=torch.float64)
    R   = torch.tensor(rho,    dtype=torch.float64)
    varG = (vec @ (R @ vec)) / (N_ASSETS**2)
    sigmaG = math.sqrt(varG.item())
    G0 = float(np.prod(S0)**(1/N_ASSETS))
    d1 = (math.log(G0/K) + (r + 0.5*sigmaG**2)*T) / (sigmaG*math.sqrt(T))
    d2 = d1 - sigmaG*math.sqrt(T)
    N  = lambda x: 0.5*(1+math.erf(x/math.sqrt(2)))
    return G0*N(d1) - K*math.exp(-r*T)*N(d2)

# Streaming Monte Carlo with regression CV

def price_mc_stream(params, n_paths, n_steps):
    per_gpu = n_paths // NGPU
    # running sums
    cnt = 0
    SP, SP2, SC, SC2, SPC = 0.0, 0.0, 0.0, 0.0, 0.0

    for dev_idx, dev in enumerate(DEVICES):
        engine = SobolEngine(N_ASSETS*n_steps, scramble=True, seed=SEED_BASE+dev_idx)
        for offset in range(0, per_gpu, CHUNK_PATH):
            m = min(CHUNK_PATH, per_gpu - offset)
            pay, geo = chunk_payoffs(params, m, n_steps, engine, dev)
            # update sums
            SP  += pay.sum().item()
            SP2 += (pay*pay).sum().item()
            SC  += geo.sum().item()
            SC2 += (geo*geo).sum().item()
            SPC += (pay*geo).sum().item()
            cnt += m

    # moments
    EP  = SP / cnt
    EC  = SC / cnt
    VarP = SP2/cnt - EP*EP
    VarC = SC2/cnt - EC*EC
    CovPC= SPC/cnt - EP*EC
    beta = CovPC / (VarC + 1e-12)
    # corrected price
    E_geo = geo_call_price(params['S0'], params['K'], params['r'], params['T'], params['sigma'], params['rho'])
    price_cv = EP + beta * (E_geo - EC)
    # se
    VarPCV = VarP + beta*beta*VarC - 2*beta*CovPC
    se = math.sqrt(VarPCV / cnt)
    return price_cv, se

# Demo
if __name__ == "__main__":
    import numpy as np, torch
    np.random.seed(SEED_BASE)
    torch.manual_seed(SEED_BASE)
    params = fg_sample()
    price, se = price_mc_stream(params, n_paths=100_000_000, n_steps=64)
    print(f"price = {price:.6f}, SE = {se:.6f}")


price = 56.346448, SE = 0.002206


In [3]:
import pandas as pd

x = pd.read_parquet("Train_FGAD.parquet")

x.transpose()

,0
S0_0,2.113525e+02
S0_1,1.538592e+02
S0_2,2.279245e+02
sigma_0,1.560186e-01
sigma_1,1.559945e-01
sigma_2,5.808361e-02
corr_0_1,5.957953e-01
corr_0_2,2.674293e-01
corr_1_2,4.049550e-01
K,1.000000e+02


In [5]:
import pandas as pd

x = pd.read_parquet("Train_Fast2nd.parquet")

x.transpose()

,0
S0_0,211.352480
S0_1,153.859221
S0_2,227.924505
sigma_0,0.156019
sigma_1,0.155995
sigma_2,0.058084
corr_0_1,0.595795
corr_0_2,0.267429
corr_1_2,0.404955
K,100.000000
